In [1]:
!pip install accelerate bitsandbytes datasets peft torch torchvision transformers trl==0.26.0 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 46.4 MB/s eta 0:00:00


In [2]:
from stages import PPO, SFT, Reward
from utils import generate_model_responses
import wandb

In [3]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: os_j_es (os_j_es-byu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# 1) Supervised Fine Tuning

In [4]:
sft = SFT(for_ppo=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [5]:
SAMPLE = 1000
EPOCHS = 10
BATCH = 4

In [6]:
sft_model, metrics = sft.train(data_subsample_size=SAMPLE,
                               num_epochs=EPOCHS,
                               batch_size=BATCH,
                               project_name = 'RLHF')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50257}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,11.889700
100,10.585900
150,9.669600
200,8.446200
250,7.352500
300,6.292300
350,5.202900
400,4.036800
450,2.797000
500,1.579300


train/entropy,▇███████▇▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▃▄▇█████▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▇▇▆▅▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/mean_token_accuracy,▁▁▁▁▂▇██████████████████████████████████
train/num_tokens,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
total_flos,2351670755328000.0
train/entropy,0.44712
train/epoch,10


# 2) Reward Model Fitting

In [7]:
reward = Reward(max_seq_len=1024)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
SAMPLE = 1000
EPOCHS = 10
BATCH = 4

In [9]:
reward_model, r_metrics = reward.train(num_epochs = EPOCHS,
                                       batch_size = BATCH,
                                       subsample_size = SAMPLE,
                                       project_name = 'RLHF')

README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Filtering eval >1024 tokens:   0%|          | 0/100 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Num Tokens,Min Reward,Mean Reward,Max Reward,Accuracy,Margin
1,0.697500,0.693843,393399.000000,3.880709,4.021137,4.188101,0.490385,-0.000394
2,0.670800,0.723534,786798.000000,5.245493,6.249343,8.087440,0.528846,0.038950
3,0.538900,0.891202,1180197.000000,6.900541,10.002742,14.230769,0.519231,0.133263
4,0.433000,1.055364,1573596.000000,6.707632,10.684721,16.129207,0.567308,0.303711
5,0.352100,1.422677,1966995.000000,5.951623,10.647329,18.229567,0.576923,0.380897
6,0.158500,2.072134,2360394.000000,7.121995,14.687988,21.924279,0.490385,0.533428
7,0.082100,2.438311,2753793.000000,4.763672,13.550030,22.114183,0.461538,0.508564
8,0.057700,2.426066,3147192.000000,2.998056,11.278006,21.622596,0.490385,0.663209
9,0.018800,2.726026,3540591.000000,2.847900,12.523965,22.509615,0.480769,0.728161
10,0.016500,2.828957,3933990.000000,2.643949,12.814626,22.680288,0.480769,0.713965


eval/accuracy,▃▅▅▇█▃▁▃▂▂
eval/loss,▁▁▂▂▃▆▇▇██
eval/margin,▁▁▂▄▅▆▆▇██
eval/max_reward,▁▂▅▆▆█████
eval/mean_reward,▁▂▅▅▅█▇▆▇▇
eval/min_reward,▃▅█▇▆█▄▂▁▁
eval/num_tokens,▁▂▃▃▄▅▆▆▇█
eval/runtime,█▁▁▁▂▁▁▁▁▂
eval/samples_per_second,▁███▇████▇
eval/steps_per_second,▁███▇████▇
+11,...


# 3) Optimization with PPO

In [10]:
ppo = PPO(sft_model=sft_model, reward_model=reward_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
RESP_LEN = 32
TOT_EPI = 1000
DEV_BATCH = 8
MINI_BATCH = 1
PPO_EPOCHS = 10
GRAD_ACCUM_STEP = 1

In [42]:
ppo._v_num = 0

In [12]:
ppo_model, _ = ppo.train(subsample_size=SAMPLE,
                         val_size=0.1,
                         max_resp_len = RESP_LEN,
                         total_episodes = TOT_EPI,
                         device_batch_size = DEV_BATCH,
                         num_mini_batches = MINI_BATCH,
                         num_ppo_epochs = PPO_EPOCHS,
                         grad_accum_steps = GRAD_ACCUM_STEP)

Parameter 'function'=<function PPO._load_prompt_only_dataset.<locals>.tokenize_fn at 0x7b3e7061f560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

===training policy===


Step,Training Loss


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ Provide a sensible response to a customer's │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.490006446838379  │
│ service inquiry.                            │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find a solution to the following Sudoku     │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.5709123611450195 │
│ game.                                       │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Provide an analysis of the impact of        │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.728240966796875  │
│ carbohydrates on human health.              │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find the total cost for two tickets for the │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.453135013580322  │
│ concert                                     │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Which is the most popular programming       │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.214660167694092  │
│ language as of 2021?                        │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
└─────────────────────────────────────────────┴──────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ Provide a sensible response to a customer's │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.490006446838379  │
│ service inquiry.                            │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find a solution to the following Sudoku     │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.5709123611450195 │
│ game.                                       │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Provide an analysis of the impact of        │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.728240966796875  │
│ carbohydrates on human health.              │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find the total cost for two tickets for the │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.453135013580322  │
│ concert                                     │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Which is the most popular programming       │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.214660167694092  │
│ language as of 2021?                        │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
└─────────────────────────────────────────────┴──────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ Provide a sensible response to a customer's │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.490006446838379  │
│ service inquiry.                            │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find a solution to the following Sudoku     │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.5709123611450195 │
│ game.                                       │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Provide an analysis of the impact of        │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.728240966796875  │
│ carbohydrates on human health.              │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find the total cost for two tickets for the │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.453135013580322  │
│ concert                                     │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Which is the most popular programming       │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.214660167694092  │
│ language as of 2021?                        │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
└─────────────────────────────────────────────┴──────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ Provide a sensible response to a customer's │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.490006446838379  │
│ service inquiry.                            │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find a solution to the following Sudoku     │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.5709123611450195 │
│ game.                                       │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Provide an analysis of the impact of        │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.728240966796875  │
│ carbohydrates on human health.              │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Find the total cost for two tickets for the │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.453135013580322  │
│ concert                                     │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Which is the most popular programming       │  Tart Tart Tart Tart Tart Tart Tart Tart     │ 4.214660167694092  │
│ language as of 2021?                        │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart Tart Tart Tart │                    │
│                                             │ Tart Tart Tart Tart Tart Tart                │                    │
└─────────────────────────────────────────────┴──────────────────────────────────────────────┴────────────────────┘

train/episode,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
train/eps,▁▁██████████████████████████████████████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█████
train/loss/policy_avg,▅▁▄▄▃▄▄▇█▇██████████████████████████████
train/loss/value_avg,█▄▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,█████▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁
train/objective/entropy,█▄▃▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/objective/kl,▃█▆█▃▄▅▄▄▂▂▂▁▂▂▁▂▂▁▂▂▁▃▁▁▂▂▂▁▂▂▁▂▁▂▂▃▁▂▁
train/objective/non_score_reward,▂▁▂▅▁▄▄▇▇▇▆▇█▇▇▇▇▇▇█▇█▇█▇█▇▇█▇▇███▇▆▆█▇▇
+9,...
